In [1]:
import requests

In [2]:
from openai import OpenAI

In [3]:
import json

In [4]:
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [6]:
import os

In [7]:
stock_api = os.getenv("ALPHA_VANTAGE_API_KEY")

In [8]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [9]:
from agents import Agent, Runner, function_tool, ModelSettings, SQLiteSession, trace, RunConfig, handoff

In [10]:
from functools import lru_cache

In [11]:
from datetime import datetime, timedelta

In [12]:
import time

In [44]:
FINNHUB_API_KEY = os.getenv("FINNHUB_API_KEY")

In [52]:
finnhub_client = finnhub.Client(api_key = FINNHUB_API_KEY)

In [13]:
import finnhub

In [41]:
load_dotenv()

True

In [14]:
url = "https://www.alphavantage.co/query?"

In [15]:
import json

In [16]:
@function_tool
def find_ticker(text: str) -> str:
    """Based on the natural language query, use this function to find the ticker symbol"""

    # params = {
    #     "function": "SYMBOL_SEARCH",
    #     "keywords": text,
    #     "apikey": stock_api
    # }

    # response = requests.get(url, params)

    # data = response.json()

    # ticker = (data.get('bestMatches')[0]).get('1. symbol')

    return _find_ticker(text)

In [17]:
@lru_cache(maxsize = 100)
def _find_ticker(text: str) -> str:
    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": text,
        "apikey": stock_api
    }

    response = requests.get(url, params)

    data = response.json()

    ticker = data.get('bestMatches')

    filter_region = []

    for t in ticker:
        if t.get('4. region') == 'United States':
            filter_region.append(t)

    return filter_region[0].get('1. symbol')

In [16]:
_find_ticker('Microsoft')

'MSFT'

In [18]:
@function_tool
def get_income_statement(ticker: str) -> str:
    """Based on the ticker you can find the income statement of the company from this
    function. This will fetch both quarterly and annual income statements.
    """

    return _get_income_statement(ticker)

In [19]:
@lru_cache(maxsize = 100)
def _get_income_statement(ticker: str) -> str:
    """Based on the ticker you can find the income statement of the company from this
    function. This will fetch both quarterly and annual income statements.
    """

    params = {
        "function": "INCOME_STATEMENT",
        "symbol": ticker,
        "apikey": stock_api
    }

    income_statement = requests.get(url,params)

    income_statement_data = income_statement.json()

    annual_data = income_statement_data.get('annualReports')

    quarterly_data = income_statement_data.get('quarterlyReports')

    filtered_data_annual = []

    filtered_data_quarterly = []

    for report in annual_data:
        filtered_data_annual.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'grossProfit': report.get('grossProfit'),
            'totalRevenue': report.get('totalRevenue'),
            'costOfRevenue': report.get('costOfRevenue'),
            'costofGoodsAndServicesSold': report.get('costofGoodsAndServicesSold'),
            'operatingIncome': report.get('operatingIncome'),
            'operatingExpenses': report.get('operatingExpenses'),
            'researchAndDevelopment': report.get('researchAndDevelopment'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'netInterestIncome': report.get('netInterestIncome'),
            'interestIncome': report.get('interestIncome'),
            'depreciationAndAmortization': report.get('depreciationAndAmortization'),
            'incomeBeforeTax': report.get('incomeBeforeTax'),
            'incomeTaxExpense': report.get('incomeTaxExpense'),
            'ebit': report.get('ebit'),
            'ebitda': report.get('ebitda'),
            'netIncome': report.get('netIncome'),
        })

    for report_quarterly in quarterly_data:
        filtered_data_quarterly.append({
            'fiscalDateEnding': report_quarterly.get('fiscalDateEnding'),
            'grossProfit': report_quarterly.get('grossProfit'),
            'totalRevenue': report_quarterly.get('totalRevenue'),
            'costOfRevenue': report_quarterly.get('costOfRevenue'),
            'costofGoodsAndServicesSold': report_quarterly.get('costofGoodsAndServicesSold'),
            'operatingIncome': report_quarterly.get('operatingIncome'),
            'operatingExpenses': report_quarterly.get('operatingExpenses'),
            'researchAndDevelopment': report_quarterly.get('researchAndDevelopment'),
            'sellingGeneralAndAdministrative': report_quarterly.get('sellingGeneralAndAdministrative'),
            'netInterestIncome': report_quarterly.get('netInterestIncome'),
            'interestIncome': report_quarterly.get('interestIncome'),
            'depreciationAndAmortization': report_quarterly.get('depreciationAndAmortization'),
            'incomeBeforeTax': report_quarterly.get('incomeBeforeTax'),
            'incomeTaxExpense': report_quarterly.get('incomeTaxExpense'),
            'ebit': report_quarterly.get('ebit'),
            'ebitda': report_quarterly.get('ebitda'),
            'netIncome': report_quarterly.get('netIncome'),
        })

    final_data = {
        "annual_data": filtered_data_annual,
        "quarterly_data": filtered_data_quarterly[:8]
    }

    return final_data

In [206]:
_get_income_statement('TSLA')

{'annual_data': [{'fiscalDateEnding': '2024-12-31',
   'grossProfit': '17450000000',
   'totalRevenue': '97690000000',
   'costOfRevenue': '80240000000',
   'costofGoodsAndServicesSold': '80240000000',
   'operatingIncome': '7076000000',
   'operatingExpenses': '10374000000',
   'researchAndDevelopment': '4540000000',
   'sellingGeneralAndAdministrative': '5150000000',
   'netInterestIncome': '1219000000',
   'interestIncome': '1569000000',
   'depreciationAndAmortization': '5368000000',
   'incomeBeforeTax': '8990000000',
   'incomeTaxExpense': '1837000000',
   'ebit': '9340000000',
   'ebitda': '14708000000',
   'netIncome': '7130000000'},
  {'fiscalDateEnding': '2023-12-31',
   'grossProfit': '17660000000',
   'totalRevenue': '96773000000',
   'costOfRevenue': '79113000000',
   'costofGoodsAndServicesSold': '79113000000',
   'operatingIncome': '8891000000',
   'operatingExpenses': '8769000000',
   'researchAndDevelopment': '3969000000',
   'sellingGeneralAndAdministrative': '4800000

In [20]:
@function_tool
def get_annual_income_statement(json_file):

    """ 
    You are supposed to use this function, if the query is related to finding annual income statement.
    """

    annual_filter = json_file.get('annualReports')

    filtered_data = []

    for report in annual_filter:
        filtered_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'grossProfit': report.get('grossProfit'),
            'totalRevenue': report.get('totalRevenue'),
            'costOfRevenue': report.get('costOfRevenue'),
            'costofGoodsAndServicesSold': report.get('costofGoodsAndServicesSold'),
            'operatingIncome': report.get('operatingIncome'),
            'operatingExpenses': report.get('operatingExpenses'),
            'researchAndDevelopment': report.get('researchAndDevelopment'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'netInterestIncome': report.get('netInterestIncome'),
            'interestIncome': report.get('interestIncome'),
            'depreciationAndAmortization': report.get('depreciationAndAmortization'),
            'incomeBeforeTax': report.get('incomeBeforeTax'),
            'incomeTaxExpense': report.get('incomeTaxExpense'),
            'ebit': report.get('ebit'),
            'ebitda': report.get('ebitda'),
            'netIncome': report.get('netIncome'),
        })

    return filtered_data

In [101]:
get_annual_income_statement(cached_income_statement)

[{'fiscalDateEnding': '2024-12-31',
  'grossProfit': '17450000000',
  'totalRevenue': '97690000000',
  'costOfRevenue': '80240000000',
  'costofGoodsAndServicesSold': '80240000000',
  'operatingIncome': '10374000000',
  'researchAndDevelopment': '4540000000',
  'sellingGeneralAndAdministrative': '5150000000',
  'netInterestIncome': '1219000000',
  'interestIncome': '1569000000',
  'depreciationAndAmortization': '5368000000',
  'incomeBeforeTax': '8990000000',
  'incomeTaxExpense': '1837000000',
  'ebit': '9340000000',
  'ebitda': '14708000000',
  'netIncome': '7130000000'},
 {'fiscalDateEnding': '2023-12-31',
  'grossProfit': '17660000000',
  'totalRevenue': '96773000000',
  'costOfRevenue': '79113000000',
  'costofGoodsAndServicesSold': '79113000000',
  'operatingIncome': '8769000000',
  'researchAndDevelopment': '3969000000',
  'sellingGeneralAndAdministrative': '4800000000',
  'netInterestIncome': '910000000',
  'interestIncome': '1066000000',
  'depreciationAndAmortization': '46670

In [21]:
@function_tool
def get_quarterly_income_statement(json_file):

    """ 
    You are supposed to use this function, if the query is related to finding annual income statement datapoints.
    """

    quarterly_filter = json_file.get('quarterlyReports')

    filtered_qoq_data = []

    for report in quarterly_filter:
        filtered_qoq_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'grossProfit': report.get('grossProfit'),
            'totalRevenue': report.get('totalRevenue'),
            'costOfRevenue': report.get('costOfRevenue'),
            'costofGoodsAndServicesSold': report.get('costofGoodsAndServicesSold'),
            'operatingIncome': report.get('operatingIncome'),
            'operatingExpenses': report.get('operatingExpenses'),
            'researchAndDevelopment': report.get('researchAndDevelopment'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'netInterestIncome': report.get('netInterestIncome'),
            'interestIncome': report.get('interestIncome'),
            'depreciationAndAmortization': report.get('depreciationAndAmortization'),
            'incomeBeforeTax': report.get('incomeBeforeTax'),
            'incomeTaxExpense': report.get('incomeTaxExpense'),
            'ebit': report.get('ebit'),
            'ebitda': report.get('ebitda'),
            'netIncome': report.get('netIncome'),
        })


    return filtered_qoq_data[:4]

In [22]:
@function_tool
def get_balance_sheet(ticker: str) -> str:
    """Based on the ticker you can find the balance sheet of the company from this
    function. This will fetch both annual balance sheet only.
    """

    return _get_balance_sheet(ticker)

In [23]:
@lru_cache
def _get_balance_sheet(ticker: str) -> str:

    params = {
        "function": "BALANCE_SHEET",
        "symbol": ticker,
        "apikey": stock_api
    }

    balance_sheet = requests.get(url,params)

    balance_sheet_data = balance_sheet.json()

    annual_data = balance_sheet_data.get('annualReports')

    quarterly_data = balance_sheet_data.get('quarterlyReports')

    filtered_data_annual = []

    filtered_data_quarterly = []

    for report in annual_data:
        filtered_data_annual.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'totalAssets': report.get('totalAssets'),
            'totalCurrentAssets': report.get('totalCurrentAssets'),
            'cashAndCashEquivalentsAtCarryingValue': report.get('cashAndCashEquivalentsAtCarryingValue'),
            'inventory': report.get('inventory'),
            'totalNonCurrentAssets': report.get('totalNonCurrentAssets'),
            'accumulatedDepreciationAmortizationPPE': report.get('accumulatedDepreciationAmortizationPPE'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'totalLiabilities': report.get('totalLiabilities'),
            'interestIncome': report.get('interestIncome'),
            'totalCurrentLiabilities': report.get('totalCurrentLiabilities'),
            'currentLongTermDebt': report.get('currentLongTermDebt'),
            'totalShareholderEquity': report.get('totalShareholderEquity'),
            'retainedEarnings': report.get('retainedEarnings'),
        })

    for report_quarterly in quarterly_data:
        filtered_data_quarterly.append({
            'fiscalDateEnding': report_quarterly.get('fiscalDateEnding'),
            'totalAssets': report_quarterly.get('totalAssets'),
            'totalCurrentAssets': report_quarterly.get('totalCurrentAssets'),
            'cashAndCashEquivalentsAtCarryingValue': report_quarterly.get('cashAndCashEquivalentsAtCarryingValue'),
            'inventory': report_quarterly.get('inventory'),
            'totalNonCurrentAssets': report_quarterly.get('totalNonCurrentAssets'),
            'accumulatedDepreciationAmortizationPPE': report_quarterly.get('accumulatedDepreciationAmortizationPPE'),
            'sellingGeneralAndAdministrative': report_quarterly.get('sellingGeneralAndAdministrative'),
            'totalLiabilities': report_quarterly.get('totalLiabilities'),
            'interestIncome': report_quarterly.get('interestIncome'),
            'totalCurrentLiabilities': report_quarterly.get('totalCurrentLiabilities'),
            'currentLongTermDebt': report_quarterly.get('currentLongTermDebt'),
            'totalShareholderEquity': report_quarterly.get('totalShareholderEquity'),
            'retainedEarnings': report_quarterly.get('retainedEarnings'),
        })


    final_data = {
        "annual_data": filtered_data_annual,
        "quarterly_data": filtered_data_quarterly[:8]
    }

    

    return final_data

In [201]:
_get_balance_sheet('TSLA')

{'annual_data': [{'fiscalDateEnding': '2024-12-31',
   'totalAssets': '122070000000',
   'totalCurrentAssets': '58360000000',
   'cashAndCashEquivalentsAtCarryingValue': '16139000000',
   'inventory': '12017000000',
   'totalNonCurrentAssets': '63716000000',
   'accumulatedDepreciationAmortizationPPE': 'None',
   'sellingGeneralAndAdministrative': None,
   'totalLiabilities': '48390000000',
   'interestIncome': None,
   'totalCurrentLiabilities': '28821000000',
   'currentLongTermDebt': '2343000000',
   'totalShareholderEquity': '72913000000',
   'retainedEarnings': '35209000000'},
  {'fiscalDateEnding': '2023-12-31',
   'totalAssets': '106618000000',
   'totalCurrentAssets': '49616000000',
   'cashAndCashEquivalentsAtCarryingValue': '16398000000',
   'inventory': '13626000000',
   'totalNonCurrentAssets': '57003000000',
   'accumulatedDepreciationAmortizationPPE': 'None',
   'sellingGeneralAndAdministrative': None,
   'totalLiabilities': '43009000000',
   'interestIncome': None,
   't

In [119]:
cached_balance_sheet = get_balance_sheet(ticker_symbol)

cached_balance_sheet

{'symbol': 'TSLA',
 'annualReports': [{'fiscalDateEnding': '2024-12-31',
   'reportedCurrency': 'USD',
   'totalAssets': '122070000000',
   'totalCurrentAssets': '58360000000',
   'cashAndCashEquivalentsAtCarryingValue': '16139000000',
   'cashAndShortTermInvestments': '16139000000',
   'inventory': '12017000000',
   'currentNetReceivables': '4418000000',
   'totalNonCurrentAssets': '63716000000',
   'propertyPlantEquipment': '51501000000',
   'accumulatedDepreciationAmortizationPPE': 'None',
   'intangibleAssets': '1226000000',
   'intangibleAssetsExcludingGoodwill': '1226000000',
   'goodwill': '244000000',
   'investments': 'None',
   'longTermInvestments': 'None',
   'shortTermInvestments': '20424000000',
   'otherCurrentAssets': '5362000000',
   'otherNonCurrentAssets': 'None',
   'totalLiabilities': '48390000000',
   'totalCurrentLiabilities': '28821000000',
   'currentAccountsPayable': '12474000000',
   'deferredRevenue': 'None',
   'currentDebt': 'None',
   'shortTermDebt': '32

In [24]:
@function_tool
def annual_balance_sheet(json_data):

    """ 
    You are supposed to use this function, if the query is related to finding annual balance sheet datapoints.
    """

    annual_filter = json_data.get('annualReports')

    filtered_data = []

    for report in annual_filter:
        filtered_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'totalAssets': report.get('totalAssets'),
            'totalCurrentAssets': report.get('totalCurrentAssets'),
            'cashAndCashEquivalentsAtCarryingValue': report.get('cashAndCashEquivalentsAtCarryingValue'),
            'inventory': report.get('inventory'),
            'totalNonCurrentAssets': report.get('totalNonCurrentAssets'),
            'accumulatedDepreciationAmortizationPPE': report.get('accumulatedDepreciationAmortizationPPE'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'totalLiabilities': report.get('totalLiabilities'),
            'interestIncome': report.get('interestIncome'),
            'totalCurrentLiabilities': report.get('totalCurrentLiabilities'),
            'currentLongTermDebt': report.get('currentLongTermDebt'),
            'totalShareholderEquity': report.get('totalShareholderEquity'),
            'retainedEarnings': report.get('retainedEarnings'),
            'commonStock': report.get('commonStock')
        })

    return filtered_data

In [25]:
@function_tool
def quarterly_balance_sheet(json_data):

    """ 
    You are supposed to use this function, if the query is related to finding quarterly balance sheet datapoints.
    """

    quarterly_filter = json_data.get('quarterlyReports')

    filtered_data = []

    for report in quarterly_filter:
        filtered_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'totalAssets': report.get('totalAssets'),
            'totalCurrentAssets': report.get('totalCurrentAssets'),
            'cashAndCashEquivalentsAtCarryingValue': report.get('cashAndCashEquivalentsAtCarryingValue'),
            'inventory': report.get('inventory'),
            'totalNonCurrentAssets': report.get('totalNonCurrentAssets'),
            'accumulatedDepreciationAmortizationPPE': report.get('accumulatedDepreciationAmortizationPPE'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'totalLiabilities': report.get('totalLiabilities'),
            'interestIncome': report.get('interestIncome'),
            'totalCurrentLiabilities': report.get('totalCurrentLiabilities'),
            'currentLongTermDebt': report.get('currentLongTermDebt'),
            'totalShareholderEquity': report.get('totalShareholderEquity'),
            'retainedEarnings': report.get('retainedEarnings'),
            'commonStock': report.get('commonStock')
        })

    return filtered_data

In [26]:
@function_tool
def cash_flow_statement(ticker: str) -> str:
    """Based on the ticker found out using the function find_ticker(text) you can find the balance sheet of the company from this
    function. This will fetch both quarterly and annual cash flow statement.
    """

    return _cash_flow_statement(ticker)

In [27]:
@lru_cache
def _cash_flow_statement(ticker: str) -> str:
    """Based on the ticker found out using the function find_ticker(text) you can find the balance sheet of the company from this
    function. This will fetch both quarterly and annual cash flow statement.
    """

    params = {
        "function": "CASH_FLOW",
        "symbol": ticker,
        "apikey": stock_api
    }

    cash_flow = requests.get(url,params)

    cash_flow_data = cash_flow.json()

    annual_filter = cash_flow_data.get('annualReports')

    final_cash_flow_data = []

    for cash_flow in annual_filter:
        final_cash_flow_data.append({
            'fiscalDateEnding': cash_flow.get('fiscalDateEnding'),
            'operatingCashflow': cash_flow.get('operatingCashflow'),
            'cashflowFromInvestment': cash_flow.get('cashflowFromInvestment'),
            'cashflowFromFinancing': cash_flow.get('cashflowFromFinancing'),
            'netIncome': cash_flow.get('netIncome')
        })

    return final_cash_flow_data

In [134]:
cash_flow_statement(ticker_symbol)

[{'fiscalDateEnding': '2024-12-31',
  'operatingCashflow': '14923000000',
  'cashflowFromInvestment': '-18787000000',
  'cashflowFromFinancing': '3853000000',
  'netIncome': '7153000000'},
 {'fiscalDateEnding': '2023-12-31',
  'operatingCashflow': '13256000000',
  'cashflowFromInvestment': '-15584000000',
  'cashflowFromFinancing': '2589000000',
  'netIncome': '14974000000'},
 {'fiscalDateEnding': '2022-12-31',
  'operatingCashflow': '14724000000',
  'cashflowFromInvestment': '-11973000000',
  'cashflowFromFinancing': '-3527000000',
  'netIncome': '12556000000'},
 {'fiscalDateEnding': '2021-12-31',
  'operatingCashflow': '11497000000',
  'cashflowFromInvestment': '-7868000000',
  'cashflowFromFinancing': '-5203000000',
  'netIncome': '5519000000'},
 {'fiscalDateEnding': '2020-12-31',
  'operatingCashflow': '5943000000',
  'cashflowFromInvestment': '-3132000000',
  'cashflowFromFinancing': '9973000000',
  'netIncome': '690000000'},
 {'fiscalDateEnding': '2019-12-31',
  'operatingCashflo

In [28]:
@function_tool
def price_data(ticker: str) -> str:
    """ 
    Based on the ticker, we need to find the price of the stock through this.
    """

    return _price_data(ticker)

In [29]:
@lru_cache
def _price_data(ticker: str) -> str:
    """ 
    Based on the ticker, we need to find the price of the stock through this.
    """

    params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": ticker,
        "outputsize": "compact",
        "apikey": stock_api
    }

    response = requests.get(url,params)

    response_json = response.json()

    filtered_json = response_json['Time Series (Daily)']

    close_price = []

    for date, values in filtered_json.items():
        close_price.append(
            {
                'date': date,
                'close': values['4. close']
            }
        )

    latest_price = close_price[0]

    return latest_price

In [73]:
@function_tool
def valuation_of_peers(ticker: str) -> str:
    """ 
    Based on the ticker, this function will give you various valuation metrics of peers along with the company.
    """

    return _valuation_of_peers(ticker)

In [72]:
@lru_cache
def _valuation_of_peers(ticker: str) -> str:
    """ 
    Based on the ticker, this function will give you various valuation metrics of peers along with the company.
    """

    peers = finnhub_client.company_peers(ticker)

    top_peers = peers[:4]

    top_peers_data = []

    for peer in top_peers:
        params = {
            "function": "OVERVIEW",
            "symbol": peer,
            "apikey": stock_api
        }
    
        response = requests.get(url, params)
    
        data = response.json()
        
        filtered_data = {
            'Name': data.get('Name'),
            'Symbol': data.get('Symbol'),
            'PERatio': data.get('PERatio'),
            'PEGRatio': data.get('PEGRatio'),
            'PriceToSalesRatioTTM': data.get('PriceToSalesRatioTTM'),
            'PriceToBookRatio': data.get('PriceToBookRatio'),
            'EVToRevenue': data.get('EVToRevenue'),
            'EVToEBITDA': data.get('EVToEBITDA'),
            'Sector': data.get('Sector'),
            'Industry': data.get('Industry'),
        }

        top_peers_data.append(filtered_data)

        time.sleep(1)

    return top_peers_data

In [87]:
_valuation_of_peers('GOOG')

[{'Name': 'Alphabet Inc Class A',
  'Symbol': 'GOOGL',
  'PERatio': '31.14',
  'PEGRatio': '1.669',
  'PriceToSalesRatioTTM': '9.9',
  'PriceToBookRatio': '9.83',
  'EVToRevenue': '9.7',
  'EVToEBITDA': '21.77',
  'Sector': 'COMMUNICATION SERVICES',
  'Industry': 'INTERNET CONTENT & INFORMATION'},
 {'Name': 'Meta Platforms Inc.',
  'Symbol': 'META',
  'PERatio': '28.82',
  'PEGRatio': '1.53',
  'PriceToSalesRatioTTM': '8.65',
  'PriceToBookRatio': '8.45',
  'EVToRevenue': '8.69',
  'EVToEBITDA': '16.02',
  'Sector': 'COMMUNICATION SERVICES',
  'Industry': 'INTERNET CONTENT & INFORMATION'},
 {'Name': 'Reddit, Inc.',
  'Symbol': 'RDDT',
  'PERatio': '110.96',
  'PEGRatio': 'None',
  'PriceToSalesRatioTTM': '24.07',
  'PriceToBookRatio': '17.56',
  'EVToRevenue': '22.91',
  'EVToEBITDA': '156.41',
  'Sector': 'COMMUNICATION SERVICES',
  'Industry': 'INTERNET CONTENT & INFORMATION'},
 {'Name': 'Pinterest Inc',
  'Symbol': 'PINS',
  'PERatio': '9.32',
  'PEGRatio': '0.552',
  'PriceToSalesR

In [30]:
@function_tool
def earnings_analysis(ticker: str) -> str:
    """ 
    Based on the ticker, you will find a 10-12 point analysis related to the latest earnings report. 
    You can use it in case the user wants to do some kind of qualitative analysis
    """

    return _earnings_analysis(ticker)

In [31]:
@lru_cache
def _earnings_analysis(ticker: str) -> str:
    """
    Based on the ticker, you will find a 10-12 point analysis related to the latest earnings report. 
    You can use it in case the user wants to do some kind of qualitative analysis
    """

    final_transcript = []

    final_quarter = []

    today = datetime.now()

    transcript = None

    while  transcript is None or transcript == []:

        year = today.year
    
        month = today.month
        
        if month in [1,2,3]:
            quarter = 'Q4'
        elif month in [4,5,6]:
            quarter = 'Q1'
        elif month in [7,8,9]:
            quarter = 'Q2'
        else:
            quarter = 'Q3'
    
        final = f'{year}{quarter}'
    
        params = {
            "function": "EARNINGS_CALL_TRANSCRIPT",
            "symbol": ticker,
            "apikey": stock_api,
            "quarter": final
        }

        response = requests.get(url, params)

        final_json = response.json()

        time.sleep(1)

        transcript = final_json.get('transcript')

        returned_quarter = final_json.get('quarter')

        if transcript != []:
            final_transcript = transcript
            final_quarter = returned_quarter
            break

        today = today - timedelta(days = 30)
        

    
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": f""" 
           You will be given the full earnings call transcript in {final_transcript}.

            Your task is to extract **7–10 investor-relevant insights** strictly based on what 
            the management said in the transcript.
            
            Rules you MUST follow:
            1. Use ONLY information explicitly stated by executives (CEO, CFO, or Investor Relations).  
               Do NOT include analyst opinions, assumptions, or external interpretation.
            2. Each point must represent a **distinct insight**, not a restatement of another point.
            3. For every point, clearly mention **who said it** (name + role).
            4. Do NOT use generic phrases such as “strong performance”, “positive outlook”, or “solid results” 
            unless backed by specific data or commentary.
            5. Each point must explain **why this matters to a long-term investor** 
            (growth, margins, cash flow, competitive position, or strategic direction).
            6. Prefer **synthesis over repetition** — combine numbers with management commentary where relevant.
            7. Do NOT introduce any facts, estimates, or forward views not explicitly stated in the transcript.
            
            Output format:
            - Heading: Analysis of the earnings call ({final_quarter})
            - Numbered list (7–10 points)
            - 2–4 sentences per point
            - End each point with: *(Source: Name, Title)*
            
            Tone:
            - Analytical
            - Investor-oriented
            - Neutral and factual

            Don't only include positive or negative points, try to include both
            """}
        ], temperature = 0.3
    )

    return response.choices[0].message.content

In [170]:
_earnings_analysis('AMZN')

'# Analysis of the earnings call (2025Q3)\n\n1. **AWS Revenue Growth**: Andrew Jassy, CEO, reported that AWS revenue grew by 20.2% year-over-year, reaching an annualized run rate of $132 billion. This acceleration is attributed to strong demand for both AI and core services, indicating AWS\'s competitive position in the cloud market and its ability to capitalize on the growing AI sector. This growth is crucial for long-term investors as it reflects AWS\'s resilience and potential for continued expansion in a rapidly evolving market. *(Source: Andrew Jassy, CEO)*\n\n2. **Impact of Special Charges on Operating Income**: Brian Olsavsky, CFO, noted that operating income for the quarter was $17.4 billion, which was reduced by $4.3 billion due to special charges related to a legal settlement and severance costs. Excluding these charges, operating income would have been $21.7 billion, highlighting the underlying profitability of the business. For investors, this distinction is important as it

In [102]:
_earnings_analysis('TSLA')

"# Analysis of the earnings call (['2025Q4'])\n\n1. Apple reported a revenue of $123.9 billion for the quarter, which represents a year-over-year decline of 3%. This decline was primarily attributed to lower iPhone sales, indicating potential challenges in maintaining growth in its flagship product line. Long-term investors should monitor how Apple addresses this decline to sustain revenue growth and market share. *(Source: Tim Cook, CEO)*\n\n2. The Services segment generated $22.6 billion in revenue, marking a 15% increase year-over-year. This growth in Services highlights Apple's successful strategy to diversify its revenue streams beyond hardware sales, which can enhance margins and provide a more stable cash flow. Investors should view this as a positive indicator of Apple's ability to leverage its ecosystem. *(Source: Luca Maestri, CFO)*\n\n3. Apple’s gross margin for the quarter was reported at 38.5%, down from 39.8% in the previous year. This decline in margin could signal incre

In [45]:
# global_session = SQLiteSession("test_conversation")

In [32]:
async def _agent_builder(query, workflow = "random_test"):

    agent = Agent(
        name = "Data Finder",
        model = 'gpt-4o-mini',
        instructions = f"""
        You are a stock analyser and your job is to understand the {query} and then use the tools available at your disposal 
        to answer to the user. 

        Make sure that you ONLY get data from tools and in case the query cannot be solved by tools, empathetically refuse to answer
        the user query. 

        It is recommended you use tables, if you're doing any comparison either between 2 or more companies or you're 
        showing the data of more than 1 year.

        In the tools you are now getting annual and quarterly information for {get_income_statement} & {get_balance_sheet}.

        You can use both to do your analysis now. 
        
        Quarterly data is for the last 8 quarters!

        You can use the {earnings_analysis} tool in case the user wants to know what happened in the latest earnings call of a particular
        company.

        However, you should always try to back it up with financial data first. 
        
        EXCEPTION:
        If the user is specifically asking to summarise the earning call data you are supposed to simply just print out
        the output you're getting from {earnings_analysis}.
        """,
        tools = [
            price_data,
            cash_flow_statement,
            find_ticker,
            get_balance_sheet,
            get_income_statement,
            earnings_analysis
        ],
        model_settings = ModelSettings(
            tool_choice = "auto",
            seed = 0,
            temperature = 0.3
        )
    )
    
    result = await Runner.run(agent, query, run_config = RunConfig(workflow_name = workflow))

    return print(result.final_output)

    # tools_called = {
    #     item.raw_item.name
    #     for item in result.new_items

    #     if item.type == "tool_call_item"
    # }
    # return tools_called

In [61]:
await _agent_builder("Does google have a greater revenue than microsoft?", workflow = "Income Statement Check")

To determine whether Google (Alphabet Inc.) has greater revenue than Microsoft, let's compare their total revenue for the most recent fiscal year available.

### Annual Revenue Comparison

| Company       | Fiscal Year Ending | Total Revenue (in billions) |
|---------------|--------------------|-----------------------------|
| Alphabet Inc. | 2023-12-31         | $307.39                     |
| Microsoft     | 2023-06-30         | $211.92                     |

### Conclusion

Based on the latest available data, **Alphabet Inc. (Google)** has greater revenue than **Microsoft** for the most recent fiscal year.


In [62]:
await _agent_builder("Compare QoQ revenue of Tesla and see whether it is growing or not?", workflow = "Income Statement Check")
await _agent_builder("Does google have a greater revenue than microsoft?", workflow = "Income Statement Check")

Here's the comparison of Tesla's quarterly revenue over the last two quarters to determine if it is growing:

| Quarter Ending   | Total Revenue (in billions) | QoQ Change (in billions) |
|------------------|-----------------------------|---------------------------|
| 2023-12-31       | 25.167                      | -                         |
| 2024-03-31       | 21.301                      | -3.866                    |
| 2024-06-30       | 25.500                      | +4.199                    |
| 2024-09-30       | 28.095                      | +2.595                    |

### Analysis:
- **From 2023-12-31 to 2024-03-31**: Revenue decreased by **$3.866 billion**.
- **From 2024-03-31 to 2024-06-30**: Revenue increased by **$4.199 billion**.
- **From 2024-06-30 to 2024-09-30**: Revenue further increased by **$2.595 billion**.

### Conclusion:
Tesla's revenue showed a decline in the first quarter of 2024 but has since rebounded with growth in the subsequent quarters. Overall, the tren

In [63]:
await _agent_builder("What is the breakdown of cost of Tesla, give an analysis of the same", workflow = "Income Statement Check")
await _agent_builder("Compare QoQ revenue of Tesla and see whether it is growing or not?", workflow = "Income Statement Check")
await _agent_builder("Does google have a greater revenue than microsoft?", workflow = "Income Statement Check")

### Breakdown of Costs for Tesla (TSLA)

#### Annual Income Statement Overview

| Fiscal Year Ending | Total Revenue | Cost of Revenue | Gross Profit | Operating Expenses | Net Income |
|--------------------|---------------|------------------|--------------|--------------------|------------|
| 2024-12-31         | $97,690,000,000 | $80,240,000,000  | $17,450,000,000 | $10,374,000,000    | $7,130,000,000 |
| 2023-12-31         | $96,773,000,000 | $79,113,000,000  | $17,660,000,000 | $8,769,000,000     | $14,974,000,000 |
| 2022-12-31         | $81,462,000,000 | $60,609,000,000  | $20,853,000,000 | $7,021,000,000     | $12,587,000,000 |
| 2021-12-31         | $53,823,000,000 | $40,217,000,000  | $13,606,000,000 | $7,110,000,000     | $5,644,000,000 |
| 2020-12-31         | $31,536,000,000 | $24,906,000,000  | $6,630,000,000  | $4,636,000,000     | $721,000,000  |

#### Cost Breakdown Analysis

1. **Cost of Revenue**: This is the largest expense for Tesla, consistently accounting for appr

In [64]:
await _agent_builder("Could you tell me about the cost of revenue for Tesla?", workflow = "Income Statement Check")
await _agent_builder("What is the breakdown of cost of Tesla, give an analysis of the same", workflow = "Income Statement Check")
await _agent_builder("Compare QoQ revenue of Tesla and see whether it is growing or not?", workflow = "Income Statement Check")
await _agent_builder("Does google have a greater revenue than microsoft?", workflow = "Income Statement Check")

Here's the cost of revenue for Tesla (TSLA) over the past few years, along with the total revenue for context:

| Fiscal Year Ending | Total Revenue (USD) | Cost of Revenue (USD) |
|--------------------|---------------------|------------------------|
| 2024-12-31         | 97,690,000,000      | 80,240,000,000         |
| 2023-12-31         | 96,773,000,000      | 79,113,000,000         |
| 2022-12-31         | 81,462,000,000      | 60,609,000,000         |
| 2021-12-31         | 53,823,000,000      | 40,217,000,000         |
| 2020-12-31         | 31,536,000,000      | 24,906,000,000         |
| 2019-12-31         | 24,578,000,000      | 20,509,000,000         |
| 2018-12-31         | 21,461,268,000      | 17,419,247,000         |
| 2017-12-31         | 11,759,000,000      | 9,536,000,000         |

### Summary:
- The cost of revenue has increased significantly over the years, reflecting Tesla's growth in production and sales.
- In 2024, the cost of revenue is projected to be approxima

In [65]:
await _agent_builder("Is tesla growing year on year and quarter on quarter?", workflow = "Income Statement Check")

To determine if Tesla (TSLA) is growing year-on-year (YoY) and quarter-on-quarter (QoQ), we can analyze the financial data from the latest income statements and balance sheets.

### Year-on-Year Analysis

#### Annual Income Statement Summary

| Fiscal Year Ending | Total Revenue | Gross Profit | Net Income |
|---------------------|---------------|--------------|------------|
| 2024-12-31          | $97.69B       | $17.45B      | $7.13B     |
| 2023-12-31          | $96.77B       | $17.66B      | $14.97B    |
| 2022-12-31          | $81.46B       | $20.85B      | $12.59B    |
| 2021-12-31          | $53.82B       | $13.61B      | $5.64B     |

#### Year-on-Year Growth Rates

- **Total Revenue Growth**:
  - 2024 vs 2023: \( \frac{97.69 - 96.77}{96.77} \times 100 \approx 0.95\% \)
  - 2023 vs 2022: \( \frac{96.77 - 81.46}{81.46} \times 100 \approx 18.79\% \)

- **Net Income Growth**:
  - 2024 vs 2023: \( \frac{7.13 - 14.97}{14.97} \times 100 \approx -52.45\% \)
  - 2023 vs 2022: \( \frac{

In [66]:
# Test Results:
# 1. 40% pass rate
# 2. Agent is calling balance sheet, even for basic queries that we are asking

In [68]:
await _agent_builder("Give me a breakdown of Tesla's assets & liabilities?", workflow = "Balance Sheet Check")

Here's a breakdown of Tesla's assets and liabilities based on the most recent annual data:

### Annual Balance Sheet Summary

| Fiscal Year Ending | Total Assets | Total Current Assets | Cash & Cash Equivalents | Inventory | Total Non-Current Assets | Total Liabilities | Total Current Liabilities | Current Long-Term Debt | Total Shareholder Equity | Retained Earnings |
|---------------------|--------------|----------------------|-------------------------|-----------|--------------------------|-------------------|--------------------------|-----------------------|-------------------------|-------------------|
| 2024-12-31          | $122,070,000,000 | $58,360,000,000     | $16,139,000,000        | $12,017,000,000 | $63,716,000,000         | $48,390,000,000   | $28,821,000,000         | $2,343,000,000       | $72,913,000,000       | $35,209,000,000   |
| 2023-12-31          | $106,618,000,000 | $49,616,000,000     | $16,398,000,000        | $13,626,000,000 | $57,003,000,000         | $43

In [69]:
await _agent_builder("Is Tesla's assets increasing YoY & QoQ?", workflow = "Balance Sheet Check")
await _agent_builder("Give me a breakdown of Tesla's assets & liabilities?", workflow = "Balance Sheet Check")

To determine if Tesla's assets are increasing Year-over-Year (YoY) and Quarter-over-Quarter (QoQ), let's analyze the data from the balance sheets.

### Annual Data Comparison
| Fiscal Year Ending | Total Assets (in billions) | YoY Change (in billions) |
|---------------------|---------------------------|--------------------------|
| 2024-12-31          | 122.07                    | +15.45                   |
| 2023-12-31          | 106.62                    | +24.24                   |
| 2022-12-31          | 82.34                     | +19.02                   |
| 2021-12-31          | 62.13                     | +5.65                    |
| 2020-12-31          | 52.15                     | +8.06                    |

### Quarterly Data Comparison
| Fiscal Quarter Ending | Total Assets (in billions) | QoQ Change (in billions) |
|-----------------------|---------------------------|--------------------------|
| 2025-09-30            | 133.74                    | +5.17                   

In [70]:
await _agent_builder("Compare the inventory of tesla with its current assets?", workflow = "Balance Sheet Check")
await _agent_builder("Is Tesla's assets increasing YoY & QoQ?", workflow = "Balance Sheet Check")
await _agent_builder("Give me a breakdown of Tesla's assets & liabilities?", workflow = "Balance Sheet Check")

[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: server error 503, retrying.
[non-fatal] Tracing: max retries reached, giving up on this batch.


Here's a comparison of Tesla's inventory with its current assets based on the latest available data:

### Tesla's Inventory vs. Current Assets

| Fiscal Year Ending | Inventory (in $) | Total Current Assets (in $) |
|--------------------|------------------|-----------------------------|
| 2024-12-31         | 12,017,000,000   | 58,360,000,000              |
| 2023-12-31         | 13,626,000,000   | 49,616,000,000              |
| 2022-12-31         | 12,839,000,000   | 40,917,000,000              |
| 2021-12-31         | 5,757,000,000    | 27,100,000,000              |
| 2020-12-31         | 4,101,000,000    | 26,717,000,000              |
| 2019-12-31         | 3,552,000,000    | 12,103,000,000              |

### Analysis
- **Current Assets**: This includes cash, inventory, receivables, and other short-term assets. 
- **Inventory**: Represents the goods available for sale, which is a part of the current assets.

### Key Observations
1. **Inventory Growth**: Tesla's inventory has gene

In [71]:
await _agent_builder("Are shareholders equity rising for Tesla?", workflow = "Balance Sheet Check")

To determine if Tesla's shareholders' equity is rising, let's examine the annual data for the last few years:

| Fiscal Year Ending | Total Shareholder Equity (in billions) |
|--------------------|----------------------------------------|
| 2024-12-31         | 72.91                                  |
| 2023-12-31         | 62.63                                  |
| 2022-12-31         | 44.70                                  |
| 2021-12-31         | 30.19                                  |
| 2020-12-31         | 22.23                                  |

### Analysis
- **2024**: $72.91 billion
- **2023**: $62.63 billion
- **2022**: $44.70 billion
- **2021**: $30.19 billion
- **2020**: $22.23 billion

### Conclusion
Tesla's total shareholders' equity has been consistently rising over the past few years. The increase from $62.63 billion in 2023 to $72.91 billion in 2024 indicates a positive trend in shareholders' equity.


In [72]:
await _agent_builder("Compare non current assets with current assets for tesla", workflow = "Balance Sheet Check")
await _agent_builder("Compare the inventory of tesla with its current assets?", workflow = "Balance Sheet Check")
await _agent_builder("Is Tesla's assets increasing YoY & QoQ?", workflow = "Balance Sheet Check")
await _agent_builder("Give me a breakdown of Tesla's assets & liabilities?", workflow = "Balance Sheet Check")

Here's a comparison of Tesla's non-current assets and current assets over the last few fiscal years:

| Fiscal Year Ending | Total Current Assets (USD) | Total Non-Current Assets (USD) |
|--------------------|----------------------------|---------------------------------|
| 2024-12-31         | 58,360,000,000             | 63,716,000,000                  |
| 2023-12-31         | 49,616,000,000             | 57,003,000,000                  |
| 2022-12-31         | 40,917,000,000             | 41,421,000,000                  |
| 2021-12-31         | 27,100,000,000             | 35,031,000,000                  |
| 2020-12-31         | 26,717,000,000             | 25,434,000,000                  |
| 2019-12-31         | 12,103,000,000             | 22,203,000,000                  |
| 2018-12-31         | 8,307,000,000              | 21,435,000,000                  |
| 2017-12-31         | 6,570,520,000              | 22,084,948,000                  |

### Key Observations:
- **Growth in As

In [73]:
#For balance sheet related queries: 
#We have received a 100% pass rate, the agent is calling either find ticker or get balance sheet or both, and is not calling anything else

In [74]:
#cashflow category questions

test_10 = await _agent_builder("What is Amazon's operating cash flow?", workflow = "Cashflow Check")
test_11 = await _agent_builder("Does Amazon generate positive free cash flow?", workflow = "Cashflow Check")
test_12 = await _agent_builder("What is Tesla's capital expenditure?", workflow = "Cashflow Check")

print(test_10)
print(test_11)
print(test_12)

Here is the operating cash flow for Amazon (AMZN) over the last few years:

| Fiscal Year Ending | Operating Cash Flow (in billions) |
|--------------------|-----------------------------------|
| 2024-12-31         | $115.88                           |
| 2023-12-31         | $84.95                            |
| 2022-12-31         | $46.75                            |
| 2021-12-31         | $46.33                            |
| 2020-12-31         | $66.06                            |
| 2019-12-31         | $38.51                            |
| 2018-12-31         | $30.72                            |
| 2017-12-31         | $18.37                            |

### Summary
- The operating cash flow has shown a significant increase in 2024, reaching approximately **$115.88 billion**.
- There was a notable rise from **$46.75 billion** in 2022 to **$84.95 billion** in 2023.

If you need further analysis or details, feel free to ask!
To determine if Amazon (AMZN) generates positive free cash 

In [75]:
# Cashflow statement analysis:
# 1. 33% pass rate for cashflow statement
# 2. In 2/3 cases, cash flow isn't even called

In [77]:
#Earnings Commentary Check


test_14 = await _agent_builder("What did Microsoft's management say about the future of their growth?", workflow = "Earnings Analysis")
test_15 = await _agent_builder("Has Google cloud grown in the latest quarter?", workflow = "Earnings Analysis")


print(test_13)
print(test_14)
print(test_15)

It seems there is an issue retrieving the ticker for Google (Alphabet Inc.). Unfortunately, I can't access the latest earnings call details without the ticker. 

If you have any other questions or need information on a different topic, feel free to ask!
Microsoft's management provided several insights regarding the future of their growth during the latest earnings call. Here are the key points:

| **Key Insights** | **Details** |
|------------------|-------------|
| **Cloud Revenue Growth** | Microsoft Cloud surpassed $168 billion in annual revenue, reflecting a 23% year-over-year increase. This growth is driven by innovations and customer migrations. *(Source: Satya Nadella, CEO)* |
| **Azure Performance** | Azure achieved over $75 billion in annual revenue, up 34%, with significant contributions from all workloads and new data center openings across six continents. *(Source: Satya Nadella, CEO)* |
| **Commercial Bookings** | Commercial bookings exceeded $100 billion for the first tim

In [78]:
test_14 = await _agent_builder("What happened in Alphabet Inc's latest earnings call?", workflow = "Earnings Analysis")

Here’s a summary of the key points from Alphabet Inc's latest earnings call:

| **Key Point**                          | **Details**                                                                                                                                                                                                 |
|----------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Record Revenue Achievement**         | Alphabet achieved its first-ever $100 billion quarter, doubling revenue from $50 billion five years ago. This milestone enhances investor confidence in long-term revenue growth.                             |
| **AI-Driven Growth**                   | AI is significantly impacting business results, with the Gemini app processing over 650 million monthly active users and a threefo

In [79]:
#Complex Query Analysis

test_15 = await _agent_builder("Give me a complete financial analysis of Amazon", workflow = "Complex Queries")
test_16 = await _agent_builder("Is Nvidia overvalued based on its financials?", workflow = "Complex Queries")
test_17 = await _agent_builder("Is Nvidia profitable and does it generate cash?", workflow = "Complex Queries")

print(test_15)
print(test_16)
print(test_17)

Here's a complete financial analysis of Amazon (AMZN) based on the latest available data.

### Income Statement Overview

| Fiscal Year Ending | Total Revenue | Gross Profit | Operating Income | Net Income |
|--------------------|---------------|--------------|------------------|------------|
| 2024-12-31         | $637.96B      | $311.67B     | $68.59B          | $59.25B    |
| 2023-12-31         | $574.79B      | $270.05B     | $36.85B          | $30.43B    |
| 2022-12-31         | $513.98B      | $225.15B     | $12.25B          | -$2.72B    |
| 2021-12-31         | $469.82B      | $197.48B     | $24.88B          | $33.36B    |
| 2020-12-31         | $386.06B      | $152.76B     | $22.90B          | $21.33B    |

### Balance Sheet Overview

| Fiscal Year Ending | Total Assets | Total Liabilities | Total Shareholder Equity | Cash and Cash Equivalents |
|--------------------|--------------|-------------------|--------------------------|---------------------------|
| 2024-12-31         

In [80]:
# Feedback on complex queries:
# 1. Agent can't do valuation analysis 
# 2. Agents are not calling cash flow, so probably for full company analysis it doesn't see it as very important

In [33]:
financial_agent = Agent(
    name = "Financial Statement Agent",
    model = "gpt-4o-mini",
    instructions = """ 
    You are given financial statements tools like:
    - Income Statement (get_income_statement): Contains quarterly and annual both 
    - Balance Sheet (get_balance_sheet): Contains quarterly and annual both 
    - Cash Flow Statement (cash_flow_statement): Contains annual only 

    In order to get this data you need to call the tool find_ticker, by extracting the company name and sending it to the other tools.
    """,
    tools = [
        cash_flow_statement,
        find_ticker,
        get_balance_sheet,
        get_income_statement
    ],
    model_settings = ModelSettings(
            tool_choice = "auto",
            seed = 0,
            temperature = 0.3
        )
)

In [34]:
qualitative_agent = Agent(
    name = "Qualitative Agent",
    model = "gpt-4o-mini",
    instructions = """ 
    You are given qualitative agent tools like: Earnings Analysis

    This will give you the latest quarterly calls details. Remember that you always need to give the source of who said the statement, 
    if you're referring to this agent. 
    """,
    tools = [
        earnings_analysis, 
        find_ticker
    ],
    model_settings = ModelSettings(
            tool_choice = "auto",
            seed = 0,
            temperature = 0.3
        )
)

In [35]:
triage_agent = Agent(
    name = "Triage Agent",
    model = "gpt-4o-mini",
    instructions = """ 
    You are a triage stock agent, you will get a query of the user and you need to transfer it to one of the agents. 

    You are given 2 agents:
    - Financial Agent: which will give you information related to the financial information like balance sheet, income statement etc
    - Qualitative Agent: which will give you information related to the quarterly earnings of the data
    """,
    handoffs = [financial_agent, qualitative_agent]
)

In [88]:
async def agent_triage(query, workflow = "random_test"):

    qualitative_agent = Agent(
        name = "Qualitative Agent",
        model = "gpt-4o-mini",
        instructions = """ 
        You are given qualitative agent tools like: Earnings Analysis
    
        This will give you the latest quarterly calls details. Remember that you always need to give the source of who said the statement, 
        if you're referring to this agent. 
        """,
        tools = [
            earnings_analysis, 
            find_ticker
        ],
        model_settings = ModelSettings(
                tool_choice = "auto",
                seed = 0,
                temperature = 0.3
            )
    )

    financial_agent = Agent(
        name = "Financial Statement Agent",
        model = "gpt-4o-mini",
        instructions = """ 
        You are a Financial Statement Agent. Your job is to answer the user’s question using the provided tools for financial statements.
        You MUST ground all numbers in tool outputs. Do NOT guess, invent, or estimate financial values.
        
        TOOLS AVAILABLE
        - find_ticker(company_name) -> returns the best matching ticker
        - get_income_statement(ticker) -> will return both quarterly + annual. Quarters will be last 8 quarters
        - get_balance_sheet(ticker) -> will return both quarterly + annual. Quarters will be last 8 quarters
        - cash_flow_statement(ticker) -> annual only
        
        TOOL USAGE RULES (follow strictly)
        1) Always call find_ticker first using the company name inferred from the user query.
        2) Choose the minimum set of statement tools required to answer the question:
           - Profitability / growth / margins -> Income Statement
           - Liquidity / leverage / capital structure -> Balance Sheet
           - Cash generation / FCF / CFO -> Cash Flow Statement (annual only)
        3) Choose period based on the user query:
           - “quarter”, “QoQ”, “latest quarter” -> quarterly statements
           - “year”, “YoY”, “FY”, “last 5 years” -> annual statements
           - If the user doesn’t specify, default to annual for multi-year analysis and quarterly for “latest performance”.
        4) Prefer fewer tool calls over more. Do not fetch statements you won’t use.

        ANALYSIS EXPECTATIONS (don’t just list numbers)
        - For comparisons across years/quarters, present a table + 3–6 insight bullets explaining:
          a) what changed (direction + magnitude)
          b) why it matters (profitability, risk, cash, leverage)
          c) any red flags (margin compression, rising debt, cash burn, etc.)
        - When relevant, compute simple derived metrics (only from tool data):
          - Revenue growth (YoY/QoQ)
          - Gross margin, operating margin, net margin
          - Debt-to-equity (if fields exist)
          - Free cash flow = Operating Cash Flow – Capex (if both exist)
        
        """,
        tools = [
            cash_flow_statement,
            find_ticker,
            get_balance_sheet,
            get_income_statement
        ],
        model_settings = ModelSettings(
                tool_choice = "auto",
                seed = 0,
                temperature = 0.3
            )
    )

    full_analysis_agent = Agent(
        name = "Full Company Analysis",
        model = "gpt-4o-mini",
        instructions = """ 
        You are supposed to do a full analysis of a company based on the financial data and qualitative data and give the user
        a full analysis based on the financial statements as well as the earnings call data.

        Make sure you're giving sources whenever you're using earnings call data, Remember that you always need to give the 
        source of who said the statement.

        In case of comparison between 2 or more years, you can do it in a table format. And make sure you explain the data and not just 
        listing it as it is. 

        
        """,
        tools = [
            cash_flow_statement,
            find_ticker,
            get_balance_sheet,
            get_income_statement,
            earnings_analysis
        ],
        model_settings = ModelSettings(
                tool_choice = "auto",
                seed = 0,
                temperature = 0.3
            )
    )

    valuation_agent = Agent(
        name = "Valuation Analysis",
        model = "gpt-4o-mini",
        instructions = """ 
        You will be given a bunch of valuation data of the company plus its peers. 
        You need to do the analysis of the company and give a conclusion about the valuation of the company as compared to its peers. 
        First call find_ticker for the related query
        TOOL USAGE RULES (follow strictly)
        1) Always call find_ticker first using the company name inferred from the user query.
        2) The call the valuation_of_peers to get the data
        3) Infer the data and give a final analysis
        """,
        tools = [valuation_of_peers, find_ticker],
        model_settings = ModelSettings(
                tool_choice = "auto",
                seed = 0,
                temperature = 0.3
            )
    )

    triage_agent = Agent(
        name = "Triage Agent",
        model = "gpt-4o-mini",
        instructions = """ 
        You are a triage stock agent, you will get a query of the user and you need to transfer it to one of the agents. 
        
        You are given 2 agents:
        - Financial Agent: which will give you information related to the financial information like balance sheet, income statement etc
        - Qualitative Agent: which will give you information about the earnings call that happens every quarter, and will give you information
        about how the management is thinking
        - Valuation Agent: which gives valuation data of the company as compared to its peers. 
        """,
        handoffs = [financial_agent, qualitative_agent, full_analysis_agent, valuation_agent]
    )


    # result = await Runner.run(triage_agent, query, run_config = RunConfig(workflow_name = workflow))

    result = Runner.run_streamed(triage_agent, query)

    async for event in result.stream_events():
        if event.type == "raw_response_event":
            continue
        elif event.type == "agent_updated_stream_event":
            if event.new_agent.name == "Triage Agent":
                continue
            else:
                print(f"Calling {event.new_agent.name}")
            continue
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                tool_name = event.item.raw_item.name
                if tool_name == "find_ticker":
                    print(f"Finding relavant tickers for your query")
                elif tool_name == "get_income_statement":
                    print(f"Finding income statement of your stock")
                elif tool_name == "get_balance_sheet":
                    print(f"Finding balance sheet of your stock")
                elif tool_name == "cash_flow_statement":
                    print(f"Finding cash flow statement of your stock")
                elif tool_name == "earnings_analysis":
                    print(f"Pulling latest quarterly earnings report of your company")
                else:
                    pass
                
            else:
                pass

    return result.final_output

In [89]:
await agent_triage("Is Google overvalued or undervalued as compared to its peers?")

Calling Valuation Analysis
Finding relavant tickers for your query
Finding relavant tickers for your query
Finding relavant tickers for your query


"### Valuation Analysis of Alphabet Inc. (GOOGL)\n\n#### Valuation Metrics Comparison\n\n| Metric                       | Alphabet Inc. (GOOGL) | Meta Platforms Inc. (META) | Reddit, Inc. (RDDT) | Pinterest Inc. (PINS) |\n|------------------------------|-----------------------|-----------------------------|----------------------|-----------------------|\n| **P/E Ratio**                | 31.14                 | 28.82                       | 110.96               | 9.32                  |\n| **PEG Ratio**                | 1.669                 | 1.53                        | N/A                  | 0.552                 |\n| **Price to Sales Ratio (TTM)** | 9.9                  | 8.65                        | 24.07                | 4.452                 |\n| **Price to Book Ratio**      | 9.83                  | 8.45                        | 17.56                | 3.719                 |\n| **EV to Revenue**            | 9.7                   | 8.69                        | 22.91          

In [67]:
#Income Statement Analysis
test_1 = await agent_triage("Could you tell me about the cost of revenue for Tesla?")


print(test_1)

Calling Financial Statement Agent
Finding tickers for your query
Finding income statements of your stock
### Tesla, Inc. (TSLA) - Cost of Revenue Analysis

#### Annual Cost of Revenue (Last 5 Years)

| Fiscal Year Ending | Total Revenue | Cost of Revenue | Cost of Revenue as % of Revenue |
|---------------------|---------------|------------------|----------------------------------|
| 2024-12-31          | $97.69B       | $80.24B          | 82.0%                            |
| 2023-12-31          | $96.77B       | $79.11B          | 81.8%                            |
| 2022-12-31          | $81.46B       | $60.61B          | 74.4%                            |
| 2021-12-31          | $53.82B       | $40.22B          | 74.6%                            |
| 2020-12-31          | $31.54B       | $24.91B          | 79.0%                            |

### Insights

1. **Cost of Revenue Growth**: 
   - The cost of revenue has increased significantly from $24.91 billion in 2020 to $80.24 billion

In [201]:
# Income Statement Check
test_2 = await agent_triage("Give me the breakdown of Tesla's cost and give me the analysis of the same", workflow = "Income Statement Check V3")

print(test_2)

Here's a breakdown of Tesla's cost structure and a financial analysis based on the most recent annual and quarterly income statements and balance sheets.

### Income Statement Overview (Annual)

| Fiscal Year Ending | Total Revenue | Cost of Revenue | Gross Profit | Operating Income | Net Income |
|---------------------|---------------|------------------|--------------|------------------|------------|
| 2024-12-31          | $97.69B       | $80.24B          | $17.45B      | $7.08B           | $7.13B     |
| 2023-12-31          | $96.77B       | $79.11B          | $17.66B      | $8.89B           | $14.97B    |
| 2022-12-31          | $81.46B       | $60.61B          | $20.85B      | $13.66B          | $12.59B    |

### Balance Sheet Overview (Annual)

| Fiscal Year Ending | Total Assets | Total Liabilities | Total Equity | Cash & Cash Equivalents | Inventory |
|---------------------|--------------|-------------------|--------------|-------------------------|-----------|
| 2024-12-31    

In [202]:
# Income Statement Check

test_3 = await agent_triage("Compare quarter on quarter revenue of Tesla and is it growing or not?", workflow = "Income Statement Check V3")

print(test_3)

Here's a quarter-on-quarter comparison of Tesla's revenue for the latest available quarters:

| Quarter Ending | Total Revenue (in billions) | Change from Previous Quarter (in billions) |
|----------------|------------------------------|--------------------------------------------|
| 2025-09-30     | 28.095                       | +5.095                                     |
| 2025-06-30     | 22.496                       | +3.146                                     |
| 2025-03-31     | 19.335                       | +2.685                                     |
| 2024-12-31     | 25.707                       | +0.887                                     |
| 2024-09-30     | 25.182                       | +0.362                                     |
| 2024-06-30     | 25.500                       | +1.490                                     |
| 2024-03-31     | 21.301                       | +8.291                                     |
| 2023-12-31     | 25.167                       | +0

In [204]:

test_4 = await agent_triage("Does google have a greater revenue than microsoft?", workflow = "Income Statement Check V3")

print(test_4)

Here's a comparison of the annual revenues for Google (Alphabet) and Microsoft over the last few years:

### Annual Revenue Comparison (in billions)

| Fiscal Year | Google (Alphabet) | Microsoft |
|-------------|-------------------|-----------|
| 2024        | 281.72            | 350.02    |
| 2023        | 211.92            | 307.39    |
| 2022        | 198.27            | 282.84    |
| 2021        | 168.09            | 257.64    |
| 2020        | 143.02            | 182.53    |

### Insights

1. **Revenue Growth**:
   - **Google**: From 2020 to 2024, Alphabet's revenue grew from $143.02 billion to $281.72 billion, reflecting a significant increase of approximately 97%.
   - **Microsoft**: Over the same period, Microsoft's revenue increased from $182.53 billion to $350.02 billion, representing a growth of about 92%.

2. **Current Standings**:
   - As of the latest fiscal year (2024), Microsoft has a higher revenue ($350.02 billion) compared to Google ($281.72 billion). This indicates

In [206]:
# Income Statement Check
test_3 = await agent_triage("Is Tesla's revenue growing or not give an analysis of the same?", workflow = "Income Statement Check V3")

print(test_3)

### Tesla's Revenue Analysis

#### Revenue Growth Overview

| Fiscal Year Ending | Total Revenue (in $) | Year-over-Year Growth (%) |
|---------------------|----------------------|---------------------------|
| 2024-12-31          | 97,690,000,000       | 1.1%                      |
| 2023-12-31          | 96,773,000,000       | 18.5%                     |
| 2022-12-31          | 81,462,000,000       | 51.5%                     |
| 2021-12-31          | 53,823,000,000       | 70.0%                     |
| 2020-12-31          | 31,536,000,000       | 28.0%                     |
| 2019-12-31          | 24,578,000,000       | 0.9%                      |

#### Insights

1. **Recent Growth Trends**:
   - Tesla's revenue has shown a consistent upward trend over the past few years, with a significant increase of **51.5%** from 2022 to 2023. However, the growth rate has slowed to **1.1%** in the latest fiscal year (2024).

2. **Impact of Economic Conditions**:
   - The slowdown in revenue grow

In [208]:
#Traces Eval - Income Statement Check V3: 80% Pass Rate

#Previous Test Results:
# Test Results:
# 1. 40% pass rate
# 2. Agent is calling balance sheet, even for basic queries that we are asking

In [222]:
#Balance Sheet:
test_6 = await agent_triage("Compare non current assets with current assets for tesla", workflow = "Balance Sheet Check V3")
print(test_6)

Here’s a comparison of Tesla's non-current assets with current assets based on the latest available annual and quarterly balance sheet data.

### Balance Sheet Overview

| Fiscal Year Ending | Total Current Assets | Total Non-Current Assets |
|--------------------|----------------------|--------------------------|
| 2024-12-31         | $58,360,000,000      | $63,716,000,000          |
| 2023-12-31         | $49,616,000,000      | $57,003,000,000          |
| 2022-12-31         | $40,917,000,000      | $41,421,000,000          |
| 2021-12-31         | $27,100,000,000      | $35,031,000,000          |
| 2020-12-31         | $26,717,000,000      | $25,434,000,000          |

### Insights

1. **Growth in Assets**:
   - **2024 vs. 2023**: Total current assets increased by approximately **$8.74 billion** (17.6%), while total non-current assets increased by about **$6.71 billion** (11.8%). This indicates a strong growth in both asset categories, reflecting Tesla's expansion and investment in

In [223]:
#Balance Sheet:
test_7 = await agent_triage("Compare the inventory of tesla with its current assets?", workflow = "Balance Sheet Check V3")
print(test_7)

Here's a comparison of Tesla's inventory with its current assets based on the latest annual and quarterly financial statements.

### Inventory vs. Current Assets

| Fiscal Year Ending | Inventory (in $) | Total Current Assets (in $) |
|---------------------|------------------|------------------------------|
| 2024-12-31          | 12,017,000,000   | 58,360,000,000               |
| 2023-12-31          | 13,626,000,000   | 49,616,000,000               |
| 2022-12-31          | 12,839,000,000   | 40,917,000,000               |

### Insights

1. **Inventory Decrease**: 
   - From 2023 to 2024, Tesla's inventory decreased from $13.63 billion to $12.02 billion, a reduction of approximately 11.8%. This could indicate improved inventory management or a decrease in production levels.

2. **Current Assets Growth**: 
   - Total current assets increased significantly from $49.62 billion in 2023 to $58.36 billion in 2024, a growth of about 17.6%. This increase suggests that Tesla is enhancing its 

In [224]:
#Balance Sheet:
test_8 = await agent_triage("Is Tesla's assets increasing YoY & QoQ?", workflow = "Balance Sheet Check V3")
print(test_8)

### Tesla's Assets: Year-over-Year (YoY) and Quarter-over-Quarter (QoQ) Analysis

#### Summary of Total Assets

| Fiscal Year Ending | Total Assets (in billions) | Change YoY (in billions) | Change QoQ (in billions) |
|---------------------|---------------------------|--------------------------|---------------------------|
| 2024-12-31          | 122.07                    | +15.45                   | +3.10                     |
| 2023-12-31          | 106.62                    | +28.24                   | +6.45                     |
| 2022-12-31          | 82.34                     | +19.02                   | +6.01                     |
| 2021-12-31          | 62.13                     | +40.65                   | +5.88                     |
| 2020-12-31          | 52.15                     | +8.93                    | +8.51                     |

### Insights

1. **Consistent Growth**: Tesla's total assets have shown a consistent upward trend over the years, increasing from $52.15 bi

In [225]:
#Balance Sheet:
test_9 = await agent_triage("Give me a breakdown of Tesla's assets & liabilities?", workflow = "Balance Sheet Check V3")
print(test_9)

Here's a breakdown of Tesla's assets and liabilities based on the latest available annual and quarterly data.

### Annual Balance Sheet Data (Last 3 Years)

| Fiscal Year Ending | Total Assets | Total Current Assets | Total Liabilities | Total Current Liabilities | Total Shareholder Equity |
|---------------------|--------------|----------------------|-------------------|---------------------------|--------------------------|
| 2024-12-31          | $122.07B     | $58.36B              | $48.39B           | $28.82B                   | $72.91B                  |
| 2023-12-31          | $106.62B     | $49.62B              | $43.01B           | $28.75B                   | $63.63B                  |
| 2022-12-31          | $82.34B      | $40.92B              | $36.44B           | $26.71B                   | $44.70B                  |

### Quarterly Balance Sheet Data (Latest Quarter)

| Fiscal Quarter Ending | Total Assets | Total Current Assets | Total Liabilities | Total Current Liabiliti

In [226]:
# Traces Eval - Balance Sheet Check V3 - 100% Pass Rate

#Previous Test Results:
# Test Results:
# 1. 100% pass rate

#No change in with the previous results, which is great

In [227]:
test_10 = await agent_triage("What is Amazon's operating cash flow?", workflow = "Cashflow Check V2")
test_11 = await agent_triage("Does Amazon generate positive free cash flow?", workflow = "Cashflow Check V2")
test_12 = await agent_triage("What is Tesla's capital expenditure?", workflow = "Cashflow Check V2")

print(test_10)
print(test_11)
print(test_12)

Here are the operating cash flow figures for Amazon (AMZN) over the last few years:

| Fiscal Year Ending | Operating Cash Flow (in billions) | Net Income (in billions) |
|--------------------|-----------------------------------|--------------------------|
| 2024-12-31         | 115.88                            | 59.25                    |
| 2023-12-31         | 84.95                             | 30.43                    |
| 2022-12-31         | 46.75                             | -2.72                    |
| 2021-12-31         | 46.33                             | 33.36                    |
| 2020-12-31         | 66.06                             | 21.33                    |

### Insights:

1. **Significant Growth**: 
   - Operating cash flow increased from **$46.75 billion** in 2022 to **$84.95 billion** in 2023, a growth of approximately **81.5%**. This trend continued into 2024 with a further increase to **$115.88 billion**.

2. **Recovery from Losses**: 
   - In 2022, Amazon rep

In [228]:
# Traces Eval - Cashflow Check V2: 100% Pass Rate

# Previous Test Results:
# 1. 33% pass rate for cashflow statement
# 2. In 2/3 cases, cash flow isn't even called

In [231]:
test_14 = await agent_triage("What did Microsoft's management say about the future of their growth?", workflow = "Earnings Analysis V3")
test_15 = await agent_triage("Has Google cloud grown in the latest quarter?", workflow = "Earnings Analysis V3")
test_13 = await agent_triage("What happened in Alphabet Inc's latest earnings call?", workflow = "Earnings Analysis V3")

print(test_13)
print(test_14)
print(test_15)

Here’s a summary of the key points from Alphabet Inc.'s latest earnings call (2025 Q3):

1. **Record Revenue Achievement**: CEO Sundar Pichai announced that Alphabet achieved its first-ever $100 billion quarter, doubling revenue from $50 billion five years ago. This growth reflects Alphabet's strong market position and ability to leverage emerging technologies, particularly AI. *(Source: Sundar Pichai, CEO)*

2. **AI-Driven Growth in Cloud Services**: Google Cloud's revenue rose by 34% to $15.2 billion, driven by demand for enterprise AI products, indicating successful investments in AI infrastructure. *(Source: Sundar Pichai, CEO)*

3. **Increased Paid Subscriptions**: Alphabet surpassed $300 million in paid subscriptions, with growth in Google One and YouTube Premium, diversifying revenue streams and reducing reliance on advertising. *(Source: Sundar Pichai, CEO)*

4. **Strong AI Adoption in Search**: CBO Philipp Schindler reported that AI features have driven significant query growt

In [232]:
#Traces Eval - Earnings Analysis V2: 100% Pass Rate
# It perfectly called only the earnings analysis
# The only thing which was missing was that in case of Google Cloud question, it should have just spoken about Google cloud but it gave
# the analysis of all the other things spoken in the quarterly earnings call as well!

# Previous Results: 66% Pass Rate
# - When asked about Google Cloud's performance, it mistook it for Google and gave the latest quarterly revenue of Google by calling 
# Income statement

In [233]:
test_15 = await agent_triage("Give me a complete financial analysis of Amazon", workflow = "Complex Queries V2")
test_16 = await agent_triage("Is Nvidia overvalued based on its financials?", workflow = "Complex Queries V2")
test_17 = await agent_triage("Is Nvidia profitable and does it generate cash?", workflow = "Complex Queries V2")

print(test_15)
print(test_16)
print(test_17)

### Full Financial Analysis of Amazon (AMZN)

#### 1. **Income Statement Overview**
The income statement provides insights into Amazon's revenue generation and profitability over the past few years.

| Fiscal Year Ending | Total Revenue | Gross Profit | Operating Income | Net Income |
|--------------------|---------------|--------------|------------------|------------|
| 2024-12-31         | $637.96B      | $311.67B     | $68.59B          | $59.25B    |
| 2023-12-31         | $574.79B      | $270.05B     | $36.85B          | $30.43B    |
| 2022-12-31         | $513.98B      | $225.15B     | $12.25B          | -$2.72B    |
| 2021-12-31         | $469.82B      | $197.48B     | $24.88B          | $33.36B    |
| 2020-12-31         | $386.06B      | $152.76B     | $22.90B          | $21.33B    |

**Analysis**:
- **Revenue Growth**: Amazon's total revenue has shown a consistent upward trend, with a significant increase from $513.98 billion in 2022 to $637.96 billion in 2024.
- **Profitabilit

In [235]:
#Traces Eval - Complex Queries V2
#Responses of new version are better and both require valuation intervention

In [82]:
@lru_cache
def valuation_of_peers_1(ticker: str) -> str:

    peers = finnhub_client.company_peers(ticker)

    top_peers = peers[:4]

    top_peers_data = []

    for peer in top_peers:
        params = {
            "function": "OVERVIEW",
            "symbol": peer,
            "apikey": stock_api
        }
    
        response = requests.get(url, params)
    
        data = response.json()
        
        filtered_data = {
            'Name': data.get('Name'),
            'Symbol': data.get('Symbol'),
            'PERatio': data.get('PERatio'),
            'PEGRatio': data.get('PEGRatio'),
            'PriceToSalesRatioTTM': data.get('PriceToSalesRatioTTM'),
            'PriceToBookRatio': data.get('PriceToBookRatio'),
            'EVToRevenue': data.get('EVToRevenue'),
            'EVToEBITDA': data.get('EVToEBITDA'),
            'Sector': data.get('Sector'),
            'Industry': data.get('Industry'),
        }

        top_peers_data.append(filtered_data)

        time.sleep(1)

    return top_peers_data

In [83]:
valuation_of_peers_1('AAPL')

[{'Name': 'Apple Inc',
  'Symbol': 'AAPL',
  'PERatio': '36.38',
  'PEGRatio': '2.743',
  'PriceToSalesRatioTTM': '9.66',
  'PriceToBookRatio': '54.31',
  'EVToRevenue': '9.73',
  'EVToEBITDA': '27.97',
  'Sector': 'TECHNOLOGY',
  'Industry': 'CONSUMER ELECTRONICS'},
 {'Name': 'Dell Technologies Inc',
  'Symbol': 'DELL',
  'PERatio': '17.09',
  'PEGRatio': '0.544',
  'PriceToSalesRatioTTM': '0.823',
  'PriceToBookRatio': '4.39',
  'EVToRevenue': '1.022',
  'EVToEBITDA': '9.71',
  'Sector': 'TECHNOLOGY',
  'Industry': 'COMPUTER HARDWARE'},
 {'Name': 'Western Digital Corporation',
  'Symbol': 'WDC',
  'PERatio': '26.44',
  'PEGRatio': '0.952',
  'PriceToSalesRatioTTM': '6.35',
  'PriceToBookRatio': '10.9',
  'EVToRevenue': '6.53',
  'EVToEBITDA': '22.44',
  'Sector': 'TECHNOLOGY',
  'Industry': 'COMPUTER HARDWARE'},
 {'Name': 'Sandisk Corp',
  'Symbol': 'SNDK',
  'PERatio': 'None',
  'PEGRatio': 'None',
  'PriceToSalesRatioTTM': '-',
  'PriceToBookRatio': '-',
  'EVToRevenue': '-',
  'EV

In [ ]:
data = 
    {'Symbol': 'GOOG',
 'AssetType': 'Common Stock',
 'Name': 'Alphabet Inc Class C',
 'Description': 'Alphabet Inc. (GOOG), headquartered in Mountain View, California, is a prominent global technology conglomerate that emerged following the restructuring of Google in October 2015. As the parent company of Google and its numerous subsidiaries, Alphabet leads in diverse sectors including search services, digital advertising, cloud computing, consumer electronics, and artificial intelligence. The company has demonstrated strong revenue growth and plays a critical role in shaping the digital economy, leveraging cutting-edge technology to drive innovation. With a seasoned leadership team dedicated to strategic expansion, Alphabet is well-positioned to capitalize on emerging opportunities within the fast-evolving technology landscape.',
 'CIK': '1652044',
 'Exchange': 'NASDAQ',
 'Currency': 'USD',
 'Country': 'USA',
 'Sector': 'COMMUNICATION SERVICES',
 'Industry': 'INTERNET CONTENT & INFORMATION',
 'Address': '1600 AMPHITHEATRE PARKWAY, MOUNTAIN VIEW, CA, UNITED STATES, 94043',
 'OfficialSite': 'https://abc.xyz',
 'FiscalYearEnd': 'December',
 'LatestQuarter': '2025-09-30',
 'MarketCapitalization': '3806490788000',
 'EBITDA': '145174004000',
 'PERatio': '31.16',
 'PEGRatio': '1.67',
 'BookValue': '32.03',
 'DividendPerShare': '1.02',
 'DividendYield': '0.0033',
 'EPS': '10.12',
 'RevenuePerShareTTM': '31.71',
 'ProfitMargin': '0.322',
 'OperatingMarginTTM': '0.305',
 'ReturnOnAssetsTTM': '0.163',
 'ReturnOnEquityTTM': '0.354',
 'RevenueTTM': '385476002000',
 'GrossProfitTTM': '228095001000',
 'DilutedEPSTTM': '10.12',
 'QuarterlyEarningsGrowthYOY': '0.353',
 'QuarterlyRevenueGrowthYOY': '0.159',
 'AnalystTargetPrice': '328.21',
 'AnalystRatingStrongBuy': '13',
 'AnalystRatingBuy': '45',
 'AnalystRatingHold': '9',
 'AnalystRatingSell': '0',
 'AnalystRatingStrongSell': '0',
 'TrailingPE': '31.16',
 'ForwardPE': '28.25',
 'PriceToSalesRatioTTM': '9.87',
 'PriceToBookRatio': '9.84',
 'EVToRevenue': '9.7',
 'EVToEBITDA': '21.77',
 'Beta': '1.07',
 '52WeekHigh': '328.46',
 '52WeekLow': '142.27',
 '50DayMovingAverage': '298.61',
 '200DayMovingAverage': '219.16',
 'SharesOutstanding': '5407000000',
 'SharesFloat': '10798396000',
 'PercentInsiders': '6.678',
 'PercentInstitutions': '60.721',
 'DividendDate': '2025-12-15',
 'ExDividendDate': '2025-12-08'}

